## Check List 4 - MotionFilter
* Motion filters are quick decision makers that prevents trying to plan a infeasible motion.
* Here, ReachChecker is tested, compared with actual planning results in random trials
*\* ~~Strikethrough~~ means no-error but none-functional cases.*
* **4-A.2 ReachChecker**  
  - pick, place, ~~sweep~~
    - single process
    - multi process
    
* You need trained model to use ReachChecker
* To use **ReachChecker**, get model files from https://github.com/rnb-disinfection/arxiv-highcap/tree/rnb-planning/model/reach_svm and put them in *$RNB_PLANNING_DIR/model/reach_svm* folder.
* For robots that are indy7 variants (indy7gripper...), copy indy7 model file and assign the robot name as the file name.

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
tip_link = crob.get_robot_tip_dict()[crob.robot_names[0]]
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries - set collision=False for all, to make reach-only problems

In [3]:
from pkg.geometry.geometry import *

floor_g = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.5,-0.2,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.5,0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=False)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=False)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepTool

In [6]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
floor_bd = pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)
goal_bd = pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))
floor_bd.check_available = lambda joint_dict: True
goal_bd.check_available = lambda joint_dict: True

# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.BOX, "gripper", link_name=tip_link, 
                                dims=(0.02,0.1,0.1), center=(0,0,0.05), rpy=(0,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True)

# add gripper fingers - By setting parent="gripper", the position of geometry can be set relative to the parent geometry
finger1 =  gscene.create_safe(GEOTYPE.BOX, "finger1", link_name=tip_link,
                              dims=(0.02,0.02,0.1), center=(0,0.06,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")
finger2 =  gscene.create_safe(GEOTYPE.BOX, "finger2", link_name=tip_link, 
                              dims=(0.02,0.02,0.1), center=(0,-0.06,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="gripper", _type=Gripper2Tool, point=(0,0,0.11), rpy=(-np.pi/2,0,0))

# Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name=tip_link, 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=True, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepTool, point=(0,0,0), rpy=(0,0,0))

##### create_subject

In [7]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepPoint
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})
## create sweep task with 2 waypoints
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )

##### prepare planner

In [8]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT
mplan = MoveitPlanner(pscene)
tplan = TaskRRT(pscene)

##### ReachChecker
- pick, place

In [9]:
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.constraint.constraint_common import *

In [10]:
# ReachChecker needs end_link_couple_dict, which is a dictionary of list of coupled link names
mfilter = ReachChecker(pscene)

##### pick & place scenarios
  - below cell will automatically generate random configurations and record the filtering algorithm and planning results

##### Test a few runs with visualization

In [11]:
VISUALIZE = True
result_dict = defaultdict(list)
N_TRIAL = 10
MP_TIMEOUT = 1

for i_count in range(N_TRIAL):
    box1.set_offset_tf(center=(np.random.rand(3)-0.5)*2, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*2*pi))
    goal.set_offset_tf(center=(np.random.rand(3)-0.5)*2, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*2*pi))
    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    initial_state = pscene.rebind_all([BindingChain('box1', 'bottom', 'floor', 'floor')], crob.home_pose)
    tplan.prepare()
    
    scenario = "pick"
    from_state=initial_state
    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    
    to_node = ("gripper", 0)
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    obj, handle, actor = to_state.binding_state["box1"].get_instance_chain(pscene)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)

    success_mfilter = mfilter.check(to_state.binding_state["box1"], Q_dict)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, subject_list=pscene.get_changing_subjects(from_state, to_state)[0], timeout=MP_TIMEOUT)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.01)
        pick_state = pscene.rebind_all(
            [to_state.binding_state[sname].get_chain() for sname in pscene.subject_name_list], LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"
    
    
    if not success:
        pscene.set_object_state(initial_state)
        continue

    scenario = "place"
    from_state=pick_state
    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    
    to_node = ("goal", 0)
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    obj, handle, actor = to_state.binding_state["box1"].get_instance_chain(pscene)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    
    success_mfilter = mfilter.check(to_state.binding_state["box1"], Q_dict)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, subject_list=pscene.get_changing_subjects(from_state, to_state)[0], timeout=MP_TIMEOUT)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.01)
    pscene.set_object_state(initial_state)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"

0/10: Result match for pick: True / True
0/10: Result match for place: False / False
1/10: Result match for pick: False / False
2/10: Result match for pick: True / True
2/10: Result match for place: False / False
3/10: Result match for pick: False / False
4/10: Result match for pick: False / False
5/10: Result match for pick: False / False
6/10: Result match for pick: True / True
6/10: Result match for place: False / False
7/10: Result match for pick: True / True
7/10: Result match for place: False / False
8/10: Result match for pick: False / False
9/10: Result match for pick: False / False


##### massively test without visualization

In [12]:
VISUALIZE = False
result_dict = defaultdict(list)
N_TRIAL = 1000
MP_TIMEOUT = 1

for i_count in range(N_TRIAL):
    box1.set_offset_tf(center=(np.random.rand(3)-0.5)*2, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*2*pi))
    goal.set_offset_tf(center=(np.random.rand(3)-0.5)*2, 
                       orientation_mat=Rot_rpy((np.random.rand(3)-0.5)*2*pi))
    initial_state = pscene.initialize_state(crob.home_pose)
    pscene.set_object_state(initial_state)
    initial_state = pscene.rebind_all([BindingChain('box1', 'bottom', 'floor', 'floor')], crob.home_pose)
    tplan.prepare()
    
    scenario = "pick"
    from_state=initial_state
    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    
    to_node = ("gripper", 0)
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    obj, handle, actor = to_state.binding_state["box1"].get_instance_chain(pscene)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)

    success_mfilter = mfilter.check(to_state.binding_state["box1"], Q_dict)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, subject_list=pscene.get_changing_subjects(from_state, to_state)[0], timeout=MP_TIMEOUT)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        pscene.set_object_state(from_state)
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.01)
        pick_state = pscene.rebind_all(
            [to_state.binding_state[sname].get_chain() for sname in pscene.subject_name_list], LastQ)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"
    
    
    if not success:
        pscene.set_object_state(initial_state)
        continue

    scenario = "place"
    from_state=pick_state
    pscene.set_object_state(from_state)
    gscene.show_pose(from_state.Q)
    
    to_node = ("goal", 0)
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    obj, handle, actor = to_state.binding_state["box1"].get_instance_chain(pscene)
    Q_dict = list2dict(from_state.Q, gscene.joint_names)
    
    success_mfilter = mfilter.check(to_state.binding_state["box1"], Q_dict)

    Traj, LastQ, error, success = mplan.plan_algorithm(
        from_state, to_state, subject_list=pscene.get_changing_subjects(from_state, to_state)[0], timeout=MP_TIMEOUT)

    result_dict[scenario].append((success_mfilter, success))
    if success:
        if VISUALIZE:
            gscene.show_motion(Traj, period=0.01)
    pscene.set_object_state(initial_state)
    print("{}/{}: Result {} for {}: {} / {}".format(i_count, N_TRIAL, "match" if success_mfilter==success else "mismatch", scenario, success_mfilter, success))
    # assert success_mfilter == success, "Failure: filter result not same with motion planning result"

0/100: Result match for pick: False / False
1/100: Result match for pick: False / False
2/100: Result match for pick: False / False
3/100: Result match for pick: False / False
4/100: Result match for pick: False / False
5/100: Result match for pick: False / False
6/100: Result match for pick: False / False
7/100: Result match for pick: False / False
8/100: Result match for pick: False / False
9/100: Result match for pick: False / False
10/100: Result match for pick: False / False
11/100: Result match for pick: False / False
12/100: Result match for pick: True / True
12/100: Result match for place: False / False
13/100: Result match for pick: False / False
14/100: Result match for pick: False / False
15/100: Result match for pick: False / False
16/100: Result match for pick: True / True
16/100: Result match for place: False / False
17/100: Result match for pick: True / True
17/100: Result match for place: False / False
18/100: Result match for pick: False / False
19/100: Result match fo

In [13]:
ACCURACY_CUT = 95
print("Accuracy cutline: {} %".format(ACCURACY_CUT))
for k, v in sorted(result_dict.items()):
    v_arr = np.array(v)
    suc_idx = np.where(v_arr[:, 1])[0]
    acc_val = np.round(np.mean(v_arr[:,0]==v_arr[:,1])*100, 1)
    acc_suc = np.round(np.mean(v_arr[suc_idx,0]==v_arr[suc_idx,1])*100, 1)
    print("="*60)
    print("{} accuracy: {} % / success accuracy: {} % / success count:{}/{}".format(k, acc_val, acc_suc, len(suc_idx), len(v_arr)))
    assert acc_val>ACCURACY_CUT and acc_suc>ACCURACY_CUT, \
            "Accuracy {}/{} is lower than {}. Consider debugging the algorithm.".format(acc_val, acc_suc, ACCURACY_CUT)
    print("""OK with "{}" scenario""".format(k))

Accuracy cutline: 95 %
pick accuracy: 99.0 % / success accuracy: 96.4 % / success count:28/100
OK with "pick" scenario
place accuracy: 100.0 % / success accuracy: 100.0 % / success count:10/28
OK with "place" scenario


##### planning: pick & place & sweep
- *NOTE*: We disabled all collision and motion constraint to check ReachChecker's functionality only.

In [14]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene, 
                                        motion_filters=[ReachChecker(pscene)]))
from pkg.planning.task.rrt import TaskRRT
ppline.set_task_planner(TaskRRT(pscene, 
                                explicit_rule=SweepLineTask.make_unstopable_rule(pscene, ["sweep"])))

###### reset object positions

In [15]:
floor_g.set_offset_tf(center=(0,0,0), orientation_mat=Rot_rpy((0,0,0)))
goal.set_offset_tf(center=(0.3,-0.4,0), orientation_mat=Rot_rpy((0,0,0)))
box1.set_offset_tf(center=(0.3,0.4,0.031), orientation_mat=Rot_rpy((0,0,0)))
gscene.update_markers_all()

initial_state = pscene.initialize_state(crob.home_pose)
pscene.set_object_state(initial_state)
initial_state = pscene.rebind_all([BindingChain('box1', 'bottom', 'floor', 'floor')], crob.home_pose)

##### - single process

In [16]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, max_solution_count=10,
              display=False, dt_vis=0.01, timeout_loop=20, multiprocess=False, timeout=1, timeout_constrained=3)
schedule = ppline.tplan.get_best_schedule()
ppline.play_schedule(schedule)

try: 0 - ('floor', 0)->('gripper', 0)
error: 2.9
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->1 (0.07/20.0 s, steps/err: 27(65.3460025787 ms)/0.00137657603578)
try: 0 - ('floor', 0)->('floor', 1)
error: 1.7
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->2 (0.11/20.0 s, steps/err: 16(32.1640968323 ms)/0.00168721460211)
try: 1 - ('gripper', 0)->('goal', 0)
error: 1.8
try transition motion
transition motion tried: True
result: 1 - ('gripper', 0)->('goal', 0) = success
branching: 1->3 (0.17/20.0 s, steps/err: 65(58.7680339813 ms)/0.00111255922321)
try: 3 - ('goal', 0)->('goal', 1)
error: 0.5
try transition motion
transition motion tried: True
result: 3 - ('goal', 0)->('goal', 1) = success
branching: 3->4 (0.26/20.0 s, steps/err: 58(88.3479118347 ms)/0.00139741250517)
try: 4 - ('goal', 1)->('goal', 2)
error: 2.9
end
constrained motion tried: True
Goal r

transition motion tried: True
result: 1 - ('gripper', 0)->('goal', 0) = success
branching: 1->26 (4.96/20.0 s, steps/err: 39(62.3190402985 ms)/0.0021692373477)
try: 26 - ('goal', 0)->('goal', 1)
error: 1.5
try transition motion
transition motion tried: True
result: 26 - ('goal', 0)->('goal', 1) = success
branching: 26->27 (5.0/20.0 s, steps/err: 14(30.4958820343 ms)/0.00159676698174)
try: 27 - ('goal', 1)->('goal', 2)
error: 2.7
joint max
constrained motion tried: False
Motion Plan Failure
result: 27 - ('goal', 1)->('goal', 2) = fail
try: 1 - ('gripper', 0)->('floor', 0)
error: 2.6
try transition motion
transition motion tried: True
result: 1 - ('gripper', 0)->('floor', 0) = success
branching: 1->28 (5.6/20.0 s, steps/err: 30(60.093164444 ms)/0.00131768992218)
try: 1 - ('gripper', 0)->('floor', 0)
Motion Filer Failure: ReachChecker
result: 1 - ('gripper', 0)->('floor', 0) = fail
try: 24 - ('goal', 0)->('goal', 1)
error: 0.58
try transition motion
transition motion tried: True
result: 2

======================= terminated 0: required answers acquired  (10.7/20.0) ===============================
========================== FINISHED (10.7 / 20.0 s) ==============================]
('floor', 0)->('floor', 1)
('floor', 1)->('floor', 2)
('floor', 2)->('gripper', 2)
('gripper', 2)->('goal', 2)
('goal', 2)->('goal', 2)


##### - multi process

In [17]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, max_solution_count=10,
              display=False, dt_vis=0.01, timeout_loop=10, multiprocess=True, timeout=1, timeout_constrained=3)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.05)

Use 18/36 agents
try: 0 - ('floor', 0)->('gripper', 0)
Motion Filer Failure: ReachChecker
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('floor', 1)
try: 0 - ('floor', 0)->('floor', 1)
error: 2.8
try transition motion
error: 1.8
try transition motion
transition motion tried: True
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->1 (0.17/10.0 s, steps/err: 26(51.8360137939 ms)/0.00151348734134)
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->2 (0.17/10.0 s, steps/err: 17(59.4048500061 ms)/0.00120865493997)
try: 0 - ('floor', 0)->('floor', 1)
error: 2.2
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
error: 2.4
try transition motion
transition motion tried: True
try: 0 - ('floor', 0)->('gripper', 0)
transition motion tried: True
error: 2.9
result: 0 - ('floor', 0)->('floor', 1) = success
try transition motion
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->3 (0.22/10.0 s, steps/err

try transition motion
branching: 5->18 (0.72/10.0 s, steps/err: 19(143.481016159 ms)/0.00148028967501)
try: 18 - ('goal', 0)->('goal', 1)
error: 2.9
try: 20 - ('goal', 0)->('goal', 1)
error: 2.3
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 5->20 (0.72/10.0 s, steps/err: 106(179.708003998 ms)/0.00180903411081)
try transition motion
transition motion tried: True
try transition motion
result: 8 - ('gripper', 0)->('goal', 0) = fail
transition motion tried: True
transition motion tried: True
try: 13 - ('floor', 1)->('floor', 2)
try transition motion
error: 2.6
result: 5 - ('gripper', 0)->('gripper', 1) = success
transition motion tried: True
branching: 0->21 (0.72/10.0 s, steps/err: 53(101.615905762 ms)/0.0014946282615)
try transition motion
try: 12 - ('gripper', 0)->('gripper', 1)
result: 14 - ('goal', 0)->('goal', 1) = success
error: 2.9
try: 23 - ('goal', 1)->('goal', 2)
transition motion tried: True
result: 8 - ('gripper', 0)->('gripper', 1) = success
error: 2.5
branchi

try: 12 - ('gripper', 0)->('goal', 0)
error: 1.6
try: 20 - ('goal', 0)->('gripper', 0)
try transition motion
error: 2.8
try transition motion
joint max
constrained motion tried: False
Motion Plan Failure
result: 23 - ('goal', 1)->('goal', 2) = fail
try: 2 - ('floor', 1)->('floor', 2)
error: 2.7
transition motion tried: True
result: 16 - ('gripper', 0)->('gripper', 1) = success
branching: 16->39 (1.06/10.0 s, steps/err: 79(100.357055664 ms)/0.00179062864199)
try: 16 - ('gripper', 0)->('floor', 0)
transition motion tried: True
Motion Filer Failure: ReachChecker
result: 12 - ('gripper', 0)->('goal', 0) = success
result: 16 - ('gripper', 0)->('floor', 0) = fail
branching: 12->40 (1.07/10.0 s, steps/err: 63(45.8040237427 ms)/0.00150340467456)
try: 40 - ('goal', 0)->('goal', 1)
transition motion tried: True
try: 0 - ('floor', 0)->('gripper', 0)
error: 2.8
try transition motion
error: 2.9
end
result: 20 - ('goal', 0)->('gripper', 0) = success
try transition motion
constrained motion tried: Tr

joint max
constrained motion tried: False
try: 41 - ('gripper', 0)->('goal', 0)
Motion Plan Failure
result: 13 - ('floor', 1)->('floor', 2) = success
Motion Filer Failure: ReachChecker
joint max
branching: 13->57 (1.4/10.0 s, steps/err: 99(466.557025909 ms)/1.69641845075e-05)
result: 26 - ('gripper', 1)->('gripper', 2) = fail
constrained motion tried: False
try: 57 - ('floor', 2)->('gripper', 2)
transition motion tried: True
error: 2.8
try transition motion
result: 41 - ('gripper', 0)->('goal', 0) = fail
try: 58 - ('goal', 1)->('goal', 2)
Motion Plan Failure
try: 47 - ('gripper', 2)->('goal', 2)
result: 56 - ('goal', 0)->('goal', 1) = success
error: 2.8
branching: 56->58 (1.43/10.0 s, steps/err: 7(33.3390235901 ms)/0.00172590299175)
error: 2.9
result: 50 - ('goal', 1)->('goal', 2) = fail
try: 19 - ('gripper', 1)->('gripper', 2)
error: 2.2
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
end
error: 2.6
transition motion tried: True
end
constrained motion tried: True
result: 57 

end
Motion Filer Failure: ReachChecker
result: 56 - ('goal', 0)->('gripper', 0) = success
try transition motion
constrained motion tried: True
Goal reached
result: 70 - ('gripper', 2)->('floor', 2) = fail
branching: 56->80 (1.77/10.0 s, steps/err: 64(48.4049320221 ms)/0.00151574833602)
Goal reached
try: 18 - ('goal', 0)->('gripper', 0)
try: 41 - ('gripper', 0)->('gripper', 1)
result: 31 - ('goal', 1)->('goal', 2) = success
branching: 31->82 (1.78/10.0 s, steps/err: 203(812.907934189 ms)/4.00057065112e-06)
try: 57 - ('floor', 2)->('gripper', 2)
joint max
error: 2.8
error: 2.9
transition motion tried: True
Motion Filer Failure: ReachChecker
try transition motion
try transition motion
result: 18 - ('goal', 0)->('gripper', 0) = fail
constrained motion tried: False
Motion Plan Failure
try: 83 - ('goal', 0)->('goal', 1)
++ adding return motion to acquired answer ++
error: 2.8
result: 41 - ('gripper', 0)->('goal', 0) = success
try: 47 - ('gripper', 2)->('floor', 2)
branching: 41->83 (1.8/10.0

constrained motion tried: True
try: 99 - ('gripper', 2)->('goal', 2)
try: 99 - ('gripper', 2)->('goal', 2)
try: 42 - ('gripper', 2)->('goal', 2)
result: 10 - ('floor', 1)->('floor', 2) = success
error: 1.3
error: 2.8
try transition motion
try transition motion
error: 2.9
Motion Filer Failure: ReachChecker
Goal reached
try transition motion
transition motion tried: True
transition motion tried: True
branching: 10->100 (2.12/10.0 s, steps/err: 89(415.73882103 ms)/2.10603797083e-05)
result: 27 - ('floor', 0)->('floor', 1) = success
result: 54 - ('floor', 2)->('gripper', 2) = success
result: 42 - ('gripper', 2)->('goal', 2) = fail
transition motion tried: True
branching: 27->102 (2.15/10.0 s, steps/err: 47(46.8828678131 ms)/0.00174012105587)
try: 100 - ('floor', 2)->('gripper', 2)
Goal reached
branching: 54->103 (2.15/10.0 s, steps/err: 20(74.718952179 ms)/0.00148456701127)
try: 103 - ('gripper', 2)->('goal', 2)
try: 44 - ('floor', 2)->('gripper', 2)
transition motion tried: True
error: 2.

branching: 27->121 (2.42/10.0 s, steps/err: 81(105.86309433 ms)/0.00173545196542)
++ adding return motion to acquired answer ++
======================= terminated 1: Stop called from other agent  (2.4/10.0) ===============================
======================= terminated 6: Stop called from other agent  (2.4/10.0) ===============================
Goal reached
======================= terminated 0: required answers acquired  (2.4/10.0) ===============================
Goal reached
joint max
constrained motion tried: False
Motion Plan Failure
result: 36 - ('gripper', 1)->('gripper', 2) = fail
======================= terminated 14: Stop called from other agent  (2.5/10.0) ===============================
======================= terminated 5: required answers acquired  (2.5/10.0) ===============================
end
constrained motion tried: True
======================= terminated 13: required answers acquired  (2.5/10.0) ===============================
result: 76 - ('floor', 1)->('floor', 2)